## Visualization of Robot missions

In [1]:
from pathlib import Path
import pandas as pd
import contextily as cx
import folium
import branca.colormap as cm

import matplotlib.pyplot as plt
import pickle
from matplotlib.ticker import FormatStrFormatter
import matplotlib.gridspec as gridspec

/tmp/ipykernel_577327/3088118494.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
input_data_folder_base = Path("../results/")
input_data_folder = Path.cwd() / input_data_folder_base
overall_collection_file_name = "overall_collection_2023.p"
key_parameter_file_name = "key_parameter_2023.p"
cluster_positions_file_name = "cluster_positions_2023.p"
print(f"Reading raw pandas files from \n    {input_data_folder}")

output_folder_base = Path("../html/")
output_image_folder = output_folder_base / Path("./images/")
print(f"Writing website information to \n    {output_folder_base}")

Reading raw pandas files from 
    /home/sebastian/Nextcloud/Projekte/2021_DAAD_Brasilien/2023/RoBiMo_Trop_DataSet/notebooks/../results
Writing website information to 
    ../html


In [3]:
df_results = pd.read_pickle(input_data_folder / overall_collection_file_name)
df_results["DayNight"] = df_results['PAR(umol m-2s-1)'] == 0 

In [4]:
df_results.groupby(["experiment_location", "DayNight"]).corresponding_meas_file.nunique().unstack().to_markdown()

'| experiment_location   |   False |   True |\n|:----------------------|--------:|-------:|\n| Balbina               |      17 |      9 |\n| Caldeirao             |      26 |    nan |\n| Iranduba              |      20 |     12 |\n| Jandira               |      26 |      8 |'

In [5]:
cluster_position_file = input_data_folder / Path(cluster_positions_file_name)
with open(cluster_position_file, 'rb') as fp:
    centroids = pickle.load(fp)
centroids


{'Balbina': array([[ -1.90938492, -59.46105017],
        [ -1.90498015, -59.46979682],
        [ -1.90367786, -59.46055068]]),
 'Jandira': array([[ -3.23674311, -60.24114295],
        [ -3.23378505, -60.24179794],
        [ -3.2354515 , -60.2408596 ],
        [ -3.2338013 , -60.24178698]]),
 'Caldeirao': array([[ -3.24645146, -60.21718762],
        [ -3.25525976, -60.21942943],
        [ -3.25665423, -60.21853891],
        [ -3.25408879, -60.21902   ],
        [ -3.2460189 , -60.21627407],
        [ -3.25595593, -60.21851002]]),
 'Iranduba': array([[ -3.29182389, -60.1638196 ],
        [ -3.29371632, -60.16349051],
        [ -3.29160827, -60.16397365],
        [ -3.29146567, -60.16393856]])}

In [6]:
map = folium.Map(location = [df_results.lat_est.mean(),  
                            df_results.lon_est.mean()], 
                            attr="Stamen Terrain",
                            tiles='Stamen Terrain', zoom_start = 8)

basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

basemaps['Google Satellite Hybrid'].add_to(map)

for key in centroids:
    map.add_child(folium.CircleMarker(location = [centroids[key][0][0],
                                                  centroids[key][0][1]],
                                    color = 'red',
                                    tooltip= f"Click to zoom to {key}",
                                    popup=f"<a href=\"https://sebastianzug.github.io/RoBiMo_Trop_DataSet/html/{key.lower()}.html\" onclick=\"window.open('https://sebastianzug.github.io/RoBiMo_Trop_DataSet/html/{key.lower()}.html', '_blank').focus();\">Zoom to {key}</a>",
                                    radius = 10, fill=True))

filename = "index.html"
html_folder = Path("../html/")
html_file = Path.cwd() / html_folder / Path(filename)
map.save(html_file)

map
